<a href="https://colab.research.google.com/github/AilingLiu/Survival_analysis/blob/master/Use_Survival_analysis_to_Estimate_Customer_Churn_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)

In [34]:
url = 'https://raw.githubusercontent.com/AilingLiu/Survival_analysis/master/Data/Telco-Customer-Churn.csv'
df = pd.read_csv(url, sep=',')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
df.info()

# Multivariate statistical modeling

Kaplan-Meier curves and logrank tests are useful only when the predictor variable is categorical (e.g.: treatment A vs treatment B; males vs females). They don’t work easily for quantitative predictors such as gene expression, weight, or age.

An alternative method is the Cox proportional hazards regression analysis, which works for both quantitative predictor variables and for categorical variables. Furthermore, the Cox regression model extends survival analysis methods to assess simultaneously the effect of several risk factors on survival time. [Referrence](http://www.sthda.com/english/wiki/cox-proportional-hazards-model)

To apply the cox model, we will need to encode the categorical variables, converting to numerical values.

In [29]:
# Check the number of levels in each categorical variable
object_cols = df.drop(columns=['customerID', 'TotalCharges']).select_dtypes(include='object').columns
unique_levels = pd.DataFrame({'unique_levels': [0]*len(object_cols)}, index=object_cols)

for col in object_cols:
  unique_levels.loc[col, 'unique_levels'] = df[col].nunique()

unique_levels

,unique_levels
gender,2
Partner,2
Dependents,2
PhoneService,2
MultipleLines,3
InternetService,3
OnlineSecurity,3
OnlineBackup,3
DeviceProtection,3
TechSupport,3


In [30]:
# get dummies, and drop the first level for each variable
df_enc = pd.get_dummies(df[object_cols.tolist()+['tenure', 'MonthlyCharges']], prefix=object_cols, columns=object_cols, drop_first=True)
df_enc.head()

,tenure,MonthlyCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,1,29.85,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,34,56.95,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
2,2,53.85,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
3,45,42.30,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,2,70.70,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1


Note, when I start to apply coxphfitter function, it failed due to non-convergence. The reason is the suspicion is high collinearity. According to [Cam Davidson Pilon](https://github.com/CamDavidsonPilon/lifelines/blob/master/examples/Customer%20Churn.ipynb), some of the features have heirarchical natures. Such as, if one doesn't have internet service, their online security is not applicable, because `no internet service'. The best way is to treat the hierachical nature by stratifying on the different services a user may have. Here I will stratifying users based on if they have internet service on not.

In [32]:
object_cols = df.drop(columns=['customerID', 'TotalCharges', 'InternetService']).select_dtypes(include='object').columns
fix_ = df[object_cols].applymap(lambda x: "No" if str(x).startswith("No ") else x)
ndf = fix_.join(df[['tenure', 'MonthlyCharges', 'InternetService']])

# get dummies, and drop the first level for each variable
df_enc = pd.get_dummies(ndf, prefix=object_cols, columns=object_cols, drop_first=True)
df_enc.head()

,tenure,MonthlyCharges,InternetService,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,1,29.85,DSL,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,34,56.95,DSL,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
2,2,53.85,DSL,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
3,45,42.30,DSL,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,2,70.70,Fiber optic,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1


In [0]:
# below code is credit from cam davidson pilon

churn_data = df.drop(columns=['customerID', 'TotalCharges']).applymap(lambda x: "No" if str(x).startswith("No ") else x)

#strata_cols = ['InternetService', 'StreamingMovies', 'StreamingTV', 'PhoneService']

ndf = pd.get_dummies(churn_data, 
                    columns=churn_data.columns.difference(['InternetService', 'tenure', 'MonthlyCharges']), 
                    drop_first=True)

In [17]:
ndf.head()

,tenure,InternetService,MonthlyCharges,Churn_Yes,Contract_One year,Contract_Two year,Dependents_Yes,DeviceProtection_Yes,MultipleLines_Yes,OnlineBackup_Yes,OnlineSecurity_Yes,PaperlessBilling_Yes,Partner_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PhoneService_Yes,SeniorCitizen_1,StreamingMovies_Yes,StreamingTV_Yes,TechSupport_Yes,gender_Male
0,1,DSL,29.85,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0
1,34,DSL,56.95,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1
2,2,DSL,53.85,1,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,1
3,45,DSL,42.30,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1
4,2,Fiber optic,70.70,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0


In [0]:
pip install lifelines

In [0]:
from lifelines import CoxPHFitter

cph = CoxPHFitter().fit(ndf, 'tenure', 'Churn_Yes', strata=['InternetService'])

In [19]:
cph

<lifelines.CoxPHFitter: fitted with 7043 total observations, 5174 right-censored observations>

In [20]:
cph.print_summary()

In [25]:
dummies = pd.get_dummies(
    churn_data[[ 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn' ]],
                drop_first=True
)
ndf = dummies.join(df[['MonthlyCharges']])

ndf.head()

,SeniorCitizen,tenure,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes,MonthlyCharges
0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,29.85
1,0,34,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,56.95
2,0,2,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1,53.85
3,0,45,1,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,42.30
4,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,70.70


In [0]:
from lifelines import CoxPHFitter

cph = CoxPHFitter().fit(ndf, 'tenure', 'Churn_Yes')

In [27]:
cph.print_summary()

cph.print_summary()